## importing libraries

In [1]:
import cv2
import email, smtplib, ssl
import getpass
import pywhatkit
import datetime
import os
import sys
import boto
import boto.ec2
import time

from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders

## Functions for mail send and whatsapp sending

In [2]:
me = input("Enter your email-address: ")
passwd = getpass.getpass(prompt='Your email password: ')
toaddr = input("Enter security admin email-address: ")

def send_mail():
    subject = "Security Alert!!!"

    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = me
    msg['To'] = toaddr
    msg.preamble = "security" 
    text= """Hey,
   This is the face we found in front of your camera.
   This is your photo. Hello, Shivansh!!!"""
    msg.attach(MIMEText(text))

    part = MIMEBase('application', "octet-stream")
    part.set_payload(open("me.jpg", "rb").read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment; filename="me.jpg"')
    msg.attach(part)

    try:
        s = smtplib.SMTP('smtp.gmail.com', 587)
        s.ehlo()
        s.starttls()
        s.ehlo()
        s.login(user = me, password = passwd)
        s.sendmail(me, toaddr, msg.as_string())
        s.quit()
        print("Email Sent!!!")
    except SMTPException as error:
          print ("Error")     
            
def whatsapp():
    t = datetime.datetime.now()
    try:
        pywhatkit.sendwhatmsg('+91XXXXXXXXX', 'This message has been sent as we have found the users face in front of our camera. Check your email for the picture of the user', t.hour, t.minute+1)
        print("WhatsApp Sent!!!")
    except:
        print("An Unexpected Error Occured!!!")

Enter your email-address: example1@gmail.com
Your email password: ········
Enter security admin email-address: example2@gmail.com


## Functions for AWS login Instance Launh, ebs launch and attach

In [13]:
AWS_ACCESS_KEY = boto.config.get('Credentials', 'aws_access_key_id')
AWS_ACCESS_SECRET_KEY = boto.config.get('Credentials', 'aws_secret_access_key')

conn = boto.ec2.connect_to_region("ap-south-1")

def aws():
    #### creating a new instance ####
    new_reservation = conn.run_instances("ami-052c08d70def0ac62",
                                         key_name="aws_putty",
                                         instance_type="t2.micro",
                                         security_group_ids=["sg-2f1dbf48"])
    instance = new_reservation.instances[0]
    
    conn.create_tags([instance.id], {"Name":"bogo-instance"})
    while instance.state == u'pending':
        print("Instance state: %s" % instance.state)
        time.sleep(10)
        instance.update()
    
    print("Instance state: %s" % instance.state)
    print("Public dns: %s" % instance.public_dns_name)
    
    #### Create a volume ####
    # create_volume(size, zone, snapshot=None, volume_type=None, iops=None)
    vol = conn.create_volume(5, "ap-south-1a")
    print('Volume Id: ', vol.id)
    
    # Add a Name tag to the new volume so we can find it.
    conn.create_tags([vol.id], {"Name":"bogo-volume"})
    
    # We can check if the volume is now ready and available:
    curr_vol = conn.get_all_volumes([vol.id])[0]
    while curr_vol.status == 'creating':
        curr_vol = conn.get_all_volumes([vol.id])[0]
        print('Current Volume Status: ', curr_vol.status)
        time.sleep(2)
    print('Current Volume Zone: ', curr_vol.zone)
    
    #### Attach a volume ####
    result = conn.attach_volume (vol.id, instance.id, "/dev/sdf")
    print('Attach Volume Result: ', result)

## Loading the face recognition model for person1

In [23]:
shiv_model = cv2.face.LBPHFaceRecognizer_create()
shiv_model.read('shiv.yml')

## Opening the camera to recognize the first person and calling mail and whatsapp function

In [30]:
import cv2
import numpy as np
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

cap = cv2.VideoCapture(0)
flag=0
k=0

while True:
    ret, frame = cap.read() 
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        results = shiv_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 85:
            cv2.putText(image, "Hey Shivansh", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            flag+=1
        
         
        else:
            
            cv2.putText(image, "I dont know,who r u?", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13:
        k=1
        break
    if flag==10:
        cv2.imwrite("me.jpg",frame)
        k=1
        break
        
cap.release()
cv2.destroyAllWindows() 
if k==1:
    send_mail()
    whatsapp()

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-30-71a9b023aed7>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Email Sent!!!
In 0 seconds web.whatsapp.com will open and after 20 seconds message will be delivered
WhatsApp Sent!!!


## Loading the face recognition model for person1

In [27]:
virat_model = cv2.face.LBPHFaceRecognizer_create()
virat_model.read('virat.yml')

In [3]:
## Opening the camera to recognize the second person and calling AWS function

In [28]:
import cv2
import numpy as np
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

cap = cv2.VideoCapture(0)
flag=0
k=0

while True:
    ret, frame = cap.read() 
    image, face = face_detector(frame)   
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = virat_model.predict(face)      
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'        
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
      
        if confidence > 85:
            cv2.putText(image, "Hey Virat Kohli!!!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )         
            flag+=1 
        else:         
            cv2.putText(image, "I dont know, who r u?", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Looking for a face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13:
        break
        
    if flag == 10:
        print("Recognized Virat Kohli!!!")
        k=1
        break
    
cap.release()
cv2.destroyAllWindows()  
if k==1:
    aws()

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-28-0122b6a9b000>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Recognized Virat Kohli!!!
Instance state: pending
Instance state: pending
Instance state: pending
Instance state: pending
Instance state: pending
Instance state: pending
Instance state: pending
Instance state: running
Public dns: ec2-13-235-248-251.ap-south-1.compute.amazonaws.com
Volume Id:  vol-06b0cfee2853d9d6f
Current Volume Status:  creating
Current Volume Status:  available
Current Volume Zone:  ap-south-1a
Attach Volume Result:  attaching
